In [1]:
#Problem Statements:
#The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command
#Thumbs up: Increase the Volume
#Thumbs down: Drecrease the Volume
#Left swipe:'Jump' backwards 10 seconds
#Right swipe: 'Jump' forward 10 seconds
#Stop: Pause the movie

#Each video is a sequence of 30 frames (or images)

#Understanding the Dataset
#The training data consists of a few hundered videos categorised into one of the five classes.
#Each video (typically 2-3 seconds long) is divided into a sequence of 30 frames(images).
#These videos have been recorded by various people performing one of the five gestures in front of a webcam
#similar to what the smart TV will use.


In [4]:
import numpy as np
import os
#from scipy.misc import imread, imresize  #deprecated
from matplotlib.pyplot import imread
#from scipy.misc import imresize
import imageio
from PIL import Image
from skimage.transform import rescale,resize,downscale_local_mean
import datetime
import os
import cv2
import abc

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)
#tf.set_random_seed(30) # deprecated

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

In [7]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
#batch_size = 10 #experiment with the batch size

In [8]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # lenght of the video frames
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray
project_folder="./Project_data"

In [9]:
# Function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

# Function to normalise the data
def normalize_data(data):
    return data/127.5-1

# Function to initialize all the batch image data and labels
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

#Function to train_load_batch_Images
def train_load_batch_images(source_path, folder_list, batch_num, batch_size, t):
    
    batch_data,batch_labels = init_batch_data(batch_size)
    # We will also build a agumented batch data
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)

    #create a list of image numbers you want to use for a particular video
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            batch_data[folder,idx] = (resized)
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))      

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
    
    batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
    batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
    return batch_data, batch_labels

#Function to Validate Load Batch Images
def val_load_batch_images(source_path, folder_list, batch_num, batch_size, t):
    
    batch_data,batch_labels = init_batch_data(batch_size)
    # We will also build a agumented batch data
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)

    #create a list of image numbers you want to use for a particular video
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        # M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            batch_data[folder,idx] = (resized)
            #batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))      

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        #batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
    
    #batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
    #batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
    return batch_data, batch_labels

#The generator function for Training Data Set
def train_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield train_load_batch_images(source_path, folder_list, batch, batch_size, t) 
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield train_load_batch_images(source_path, folder_list, num_batches, batch_size, t)
            
#The generator function for Validation Data Set         
def val_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield val_load_batch_images(source_path, folder_list, batch, batch_size, t) 
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield val_load_batch_images(source_path, folder_list, num_batches, batch_size, t)
            
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [9]:
class ModelBuilder(metaclass= abc.ABCMeta):
    
    def initialize_path(self,project_folder):
        self.train_doc = np.random.permutation(open(project_folder + '/' + 'train.csv').readlines())
        self.val_doc = np.random.permutation(open(project_folder + '/' + 'val.csv').readlines())
        self.train_path = project_folder + '/' + 'train'
        self.val_path =  project_folder + '/' + 'val'
        self.num_train_sequences = len(self.train_doc)
        self.num_val_sequences = len(self.val_doc)
        
    def initialize_image_properties(self,image_height=100,image_width=100):
        self.image_height=image_height
        self.image_width=image_width
        self.channels=3
        self.num_classes=5
        self.total_frames=30
          
    def initialize_hyperparams(self,frames_to_sample=30,batch_size=20,num_epochs=20):
        self.frames_to_sample=frames_to_sample
        self.batch_size=batch_size
        self.num_epochs=num_epochs
        
        
    def generator(self,source_path, folder_list, augment=False):
        img_idx = np.round(np.linspace(0,self.total_frames-1,self.frames_to_sample)).astype(int)
        batch_size=self.batch_size
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(t)//batch_size
        
            for batch in range(num_batches): 
                batch_data, batch_labels= self.one_batch_data(source_path,t,batch,batch_size,img_idx,augment)
                yield batch_data, batch_labels 

            remaining_seq=len(t)%batch_size
        
            if (remaining_seq != 0):
                batch_data, batch_labels= self.one_batch_data(source_path,t,num_batches,batch_size,img_idx,augment,remaining_seq)
                yield batch_data, batch_labels 
    
    
    def one_batch_data(self,source_path,t,batch,batch_size,img_idx,augment,remaining_seq=0):
    
        seq_len = remaining_seq if remaining_seq else batch_size
    
        batch_data = np.zeros((seq_len,len(img_idx),self.image_height,self.image_width,self.channels)) 
        batch_labels = np.zeros((seq_len,self.num_classes)) 
    
        if (augment): batch_data_aug = np.zeros((seq_len,len(img_idx),self.image_height,self.image_width,self.channels))

        
        for folder in range(seq_len): 
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
            for idx,item in enumerate(img_idx): 
                #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                #image_resized=imresize(image,(self.image_height,self.image_width,3))
                image_resized=cv2.resize(image,(self.image_height,self.image_width),interpolation = cv2.INTER_AREA)
            

                batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255
                batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255
                batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255
            
                if (augment):
                    shifted = cv2.warpAffine(image, 
                                             np.float32([[1, 0, np.random.randint(-30,30)],[0, 1, np.random.randint(-30,30)]]), 
                                            (image.shape[1], image.shape[0]))
                    
                    gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)

                    x0, y0 = np.argwhere(gray > 0).min(axis=0)
                    x1, y1 = np.argwhere(gray > 0).max(axis=0) 
                    
                    cropped=shifted[x0:x1,y0:y1,:]
                    
                    #image_resized=imresize(cropped,(self.image_height,self.image_width,3))
                    image_resized=cv2.resize(image,(self.image_height,self.image_width),interpolation = cv2.INTER_AREA)
                    
                    #shifted = cv2.warpAffine(image_resized, 
                    #                        np.float32([[1, 0, np.random.randint(-3,3)],[0, 1, np.random.randint(-3,3)]]), 
                    #                        (image_resized.shape[1], image_resized.shape[0]))
            
                    batch_data_aug[folder,idx,:,:,0] = (image_resized[:,:,0])/255
                    batch_data_aug[folder,idx,:,:,1] = (image_resized[:,:,1])/255
                    batch_data_aug[folder,idx,:,:,2] = (image_resized[:,:,2])/255
                
            
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
    
        if (augment):
            batch_data=np.concatenate([batch_data,batch_data_aug])
            batch_labels=np.concatenate([batch_labels,batch_labels])

        
        return(batch_data,batch_labels)
    
    
    def train_model(self, model, augment_data=False):
        train_generator = self.generator(self.train_path, self.train_doc,augment=augment_data)
        val_generator = self.generator(self.val_path, self.val_doc)

        model_name = 'model_init' + '_' + str(datetime.datetime.now()).replace(' ','').replace(':','_') + '/'
    
        if not os.path.exists(model_name):
            os.mkdir(model_name)
        
        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)
        callbacks_list = [checkpoint, LR]

        if (self.num_train_sequences%self.batch_size) == 0:
            steps_per_epoch = int(self.num_train_sequences/self.batch_size)
        else:
            steps_per_epoch = (self.num_train_sequences//self.batch_size) + 1

        if (self.num_val_sequences%self.batch_size) == 0:
            validation_steps = int(self.num_val_sequences/self.batch_size)
        else:
            validation_steps = (self.num_val_sequences//self.batch_size) + 1
    
        history=model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=self.num_epochs, verbose=1, 
                            callbacks=callbacks_list, validation_data=val_generator, 
                            validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
        return history

        
    @abc.abstractmethod
    def define_model(self):
        pass

# Model-1

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

nb_filters = [8,16,32,64]
#nb_dense = [1000, 500, 5]
nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [12]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation (Activation)      (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 60, 60, 16)    0

In [ ]:
#Experimenting with batch_size =40 and num_epocs=25

In [9]:
batch_size = 40
num_epochs = 25

train_generator = train_generator(train_path, train_doc, batch_size)
val_generator = val_generator(val_path, val_doc, batch_size)


model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]



if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  ./Project_data/train ; batch size = 40
Epoch 1/25
17/17 [==============================] - ETA: 0s - loss: 2.1598 - categorical_accuracy: 0.2413  Source path =  ./Project_data/val ; batch size = 40

Epoch 00001: saving model to model_init_2020-11-0811_04_47.013134/model-00001-2.15982-0.24133-1.56743-0.29000.h5
17/17 [==============================] - 1268s 75s/step - loss: 2.1598 - categorical_accuracy: 0.2413 - val_loss: 1.5674 - val_categorical_accuracy: 0.2900
Epoch 2/25
17/17 [==============================] - ETA: 0s - loss: 1.5578 - categorical_accuracy: 0.3069 
Epoch 00002: saving model to model_init_2020-11-0811_04_47.013134/model-00002-1.55781-0.30691-1.85629-0.23333.h5
17/17 [==============================] - 692s 41s/step - loss: 1.5578 - categorical_accuracy: 0.3069 - val_loss: 1.8563 - val_categorical_accuracy: 0.2333
Epoch 3/25
17/17 [==============================] - ETA: 0s - loss

Epoch 21/25
17/17 [==============================] - ETA: 0s - loss: 0.7270 - categorical_accuracy: 0.7439 
Epoch 00021: saving model to model_init_2020-11-0811_04_47.013134/model-00021-0.72698-0.74394-0.87686-0.66667.h5

Epoch 00021: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
17/17 [==============================] - 479s 28s/step - loss: 0.7270 - categorical_accuracy: 0.7439 - val_loss: 0.8769 - val_categorical_accuracy: 0.6667
Epoch 22/25
17/17 [==============================] - ETA: 0s - loss: 0.7175 - categorical_accuracy: 0.7405 
Epoch 00022: saving model to model_init_2020-11-0811_04_47.013134/model-00022-0.71748-0.74048-0.90798-0.65000.h5
17/17 [==============================] - 482s 28s/step - loss: 0.7175 - categorical_accuracy: 0.7405 - val_loss: 0.9080 - val_categorical_accuracy: 0.6500
Epoch 23/25
17/17 [==============================] - ETA: 0s - loss: 0.7500 - categorical_accuracy: 0.7180 
Epoch 00023: saving model to model_init_2020-11-0811_04_47

In [ ]:
#Increasing Batch Size=90 and Epochs to 30

In [10]:
batch_size = 90
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
8/8 [==============================] - ETA: 0s - loss: 0.8075 - categorical_accuracy: 0.6801 
Epoch 00001: saving model to model_init_2020-11-0811_04_47.013134/model-00001-0.80754-0.68015-0.88577-0.72500.h5
8/8 [==============================] - 231s 29s/step - loss: 0.8075 - categorical_accuracy: 0.6801 - val_loss: 0.8858 - val_categorical_accuracy: 0.7250
Epoch 2/30
8/8 [==============================] - ETA: 0s - loss: 0.7674 - categorical_accuracy: 0.7096 
Epoch 00002: saving model to model_init_2020-11-0811_04_47.013134/model-00002-0.76738-0.70956-0.94847-0.65000.h5
8/8 [==============================] - 228s 28s/step - loss: 0.7674 - categorical_accuracy: 0.7096 - val_loss: 0.9485 - val_categorical_accuracy: 0.6500
Epoch 3/30
8/8 [==============================] - ETA: 0s - loss: 0.7453 - categorical_accuracy: 0.7206 
Epoch 00003: saving model to model_init_2020-11-0811_04_47.013134/model-00003-0.74528-0.72059-0.75565-0.75000.h5
8/8 [==============================] - 2

Epoch 22/30
8/8 [==============================] - ETA: 0s - loss: 0.7067 - categorical_accuracy: 0.7132 
Epoch 00022: saving model to model_init_2020-11-0811_04_47.013134/model-00022-0.70665-0.71324-0.82174-0.77500.h5

Epoch 00022: ReduceLROnPlateau reducing learning rate to 3.051757957450718e-08.
8/8 [==============================] - 211s 26s/step - loss: 0.7067 - categorical_accuracy: 0.7132 - val_loss: 0.8217 - val_categorical_accuracy: 0.7750
Epoch 23/30
8/8 [==============================] - ETA: 0s - loss: 0.7442 - categorical_accuracy: 0.7353 
Epoch 00023: saving model to model_init_2020-11-0811_04_47.013134/model-00023-0.74416-0.73529-0.78866-0.65000.h5
8/8 [==============================] - 221s 28s/step - loss: 0.7442 - categorical_accuracy: 0.7353 - val_loss: 0.7887 - val_categorical_accuracy: 0.6500
Epoch 24/30
8/8 [==============================] - ETA: 0s - loss: 0.7777 - categorical_accuracy: 0.7059 
Epoch 00024: saving model to model_init_2020-11-0811_04_47.013134/mod

# CNN-LSTM Model

In [9]:
class CNN_LSTM(ModelBuilder):
    
    def define_model(self,lstm_cells=64,dense_neurons=64,dropout=0.25):

        model = Sequential()

        model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
  
        model.add(TimeDistributed(Flatten()))


        model.add(LSTM(lstm_cells))
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(Dropout(dropout))
        
        model.add(Dense(self.num_classes, activation='softmax'))
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [10]:
rnn_cnn1=CNN_LSTM()
rnn_cnn1.initialize_path(project_folder)
rnn_cnn1.initialize_image_properties(image_height=120,image_width=120)
rnn_cnn1.initialize_hyperparams(frames_to_sample=18,batch_size=50,num_epochs=30)
rnn_cnn1_model=rnn_cnn1.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
rnn_cnn1_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 18, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 18, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 18, 30, 30, 64)    1

In [11]:
print("Total Params:", rnn_cnn1_model.count_params())
history_model9=rnn_cnn1.train_model(rnn_cnn1_model,augment_data=True)

Total Params: 1657445
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
14/14 [==============================] - ETA: 0s - loss: 1.4110 - categorical_accuracy: 0.3899 
Epoch 00001: saving model to model_init_2020-11-0818_31_09.136730/model-00001-1.41105-0.38989-1.61809-0.30000.h5
14/14 [==============================] - 332s 24s/step - loss: 1.4110 - categorical_accuracy: 0.3899 - val_loss: 1.6181 - val_categorical_accuracy: 0.3000
Epoch 2/30
14/14 [==============================] - ETA: 0s - loss: 1.1261 - categorical_accuracy: 0.5339 
Epoch 00002: saving model to model_init_2020-11-0818_31_09.136730/model-00002-1.12613-0.53394-1.89550-0.20000.h5
14/14 [==============================] - 323s 23s/step - loss: 1.1261 - categorical_accuracy: 0.5339 - val_loss: 1.8955 - val_categorical_accuracy: 0.2000
Epoch 3/30
14/14 [==============================] - ETA: 0s - loss: 0.9634 - categorical_accuracy: 0.6229 
Epoch 00003: saving model to model_init_2020-

Epoch 21/30
14/14 [==============================] - ETA: 0s - loss: 0.2361 - categorical_accuracy: 0.9449 
Epoch 00021: saving model to model_init_2020-11-0818_31_09.136730/model-00021-0.23612-0.94495-2.84887-0.22000.h5

Epoch 00021: ReduceLROnPlateau reducing learning rate to 3.200000264769187e-07.
14/14 [==============================] - 322s 23s/step - loss: 0.2361 - categorical_accuracy: 0.9449 - val_loss: 2.8489 - val_categorical_accuracy: 0.2200
Epoch 22/30
14/14 [==============================] - ETA: 0s - loss: 0.2304 - categorical_accuracy: 0.9442 
Epoch 00022: saving model to model_init_2020-11-0818_31_09.136730/model-00022-0.23040-0.94419-2.98801-0.19000.h5
14/14 [==============================] - 323s 23s/step - loss: 0.2304 - categorical_accuracy: 0.9442 - val_loss: 2.9880 - val_categorical_accuracy: 0.1900
Epoch 23/30
14/14 [==============================] - ETA: 0s - loss: 0.2224 - categorical_accuracy: 0.9427 
Epoch 00023: saving model to model_init_2020-11-0818_31_09.

In [ ]:
#Increasing Image size and batch size

In [12]:
cnn_lstm1=CNN_LSTM()
cnn_lstm1.initialize_path(project_folder)
cnn_lstm1.initialize_image_properties(image_height=160,image_width=160)
cnn_lstm1.initialize_hyperparams(frames_to_sample=18,batch_size=60,num_epochs=30)
cnn_lstm1_model=rnn_cnn1.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
cnn_lstm1_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_16 (TimeDis (None, 18, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_17 (TimeDis (None, 18, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_18 (TimeDis (None, 18, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 18, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_20 (TimeDis (None, 18, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_21 (TimeDis (None, 18, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_22 (TimeDis (None, 18, 30, 30, 64)   

In [13]:
print("Total Params:", cnn_lstm1_model.count_params())
history_cnn_lstm1_model=rnn_cnn1.train_model(cnn_lstm1_model,augment_data=True)

Total Params: 1657445
Epoch 1/30
14/14 [==============================] - ETA: 0s - loss: 1.4692 - categorical_accuracy: 0.3620 
Epoch 00001: saving model to model_init_2020-11-0821_27_41.262860/model-00001-1.46916-0.36199-1.61130-0.28000.h5
14/14 [==============================] - 338s 24s/step - loss: 1.4692 - categorical_accuracy: 0.3620 - val_loss: 1.6113 - val_categorical_accuracy: 0.2800
Epoch 2/30
14/14 [==============================] - ETA: 0s - loss: 1.1942 - categorical_accuracy: 0.4970 
Epoch 00002: saving model to model_init_2020-11-0821_27_41.262860/model-00002-1.19420-0.49698-1.66770-0.24000.h5
14/14 [==============================] - 340s 24s/step - loss: 1.1942 - categorical_accuracy: 0.4970 - val_loss: 1.6677 - val_categorical_accuracy: 0.2400
Epoch 3/30
14/14 [==============================] - ETA: 0s - loss: 1.0286 - categorical_accuracy: 0.5814 
Epoch 00003: saving model to model_init_2020-11-0821_27_41.262860/model-00003-1.02862-0.58145-1.98773-0.21000.h5
14/14 [=

Epoch 22/30
14/14 [==============================] - ETA: 0s - loss: 0.3281 - categorical_accuracy: 0.9065 
Epoch 00022: saving model to model_init_2020-11-0821_27_41.262860/model-00022-0.32810-0.90649-2.32326-0.25000.h5
14/14 [==============================] - 334s 24s/step - loss: 0.3281 - categorical_accuracy: 0.9065 - val_loss: 2.3233 - val_categorical_accuracy: 0.2500
Epoch 23/30
14/14 [==============================] - ETA: 0s - loss: 0.3337 - categorical_accuracy: 0.9035 
Epoch 00023: saving model to model_init_2020-11-0821_27_41.262860/model-00023-0.33368-0.90347-2.19185-0.26000.h5
14/14 [==============================] - 329s 24s/step - loss: 0.3337 - categorical_accuracy: 0.9035 - val_loss: 2.1919 - val_categorical_accuracy: 0.2600
Epoch 24/30
14/14 [==============================] - ETA: 0s - loss: 0.3418 - categorical_accuracy: 0.9012 
Epoch 00024: saving model to model_init_2020-11-0821_27_41.262860/model-00024-0.34183-0.90121-2.20758-0.24000.h5
14/14 [====================

In [ ]:
#Reducing the batch Size

In [ ]:
cnn_lstm1=CNN_LSTM()
cnn_lstm1.initialize_path(project_folder)
cnn_lstm1.initialize_image_properties(image_height=160,image_width=160)
cnn_lstm1.initialize_hyperparams(frames_to_sample=18,batch_size=30,num_epochs=30)
cnn_lstm1_model=rnn_cnn1.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
cnn_lstm1_model.summary()

In [ ]:
print("Total Params:", cnn_lstm1_model.count_params())
history_cnn_lstm1_model=rnn_cnn1.train_model(cnn_lstm1_model,augment_data=True)

# CNN_GRU

In [13]:
class CNN_GRU(ModelBuilder):
    
    def define_model(self,lstm_cells=64,dense_neurons=64,dropout=0.25):

        model = Sequential()

        model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        

        model.add(TimeDistributed(Flatten()))


        model.add(GRU(lstm_cells))
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(Dropout(dropout))
        
        model.add(Dense(self.num_classes, activation='softmax'))
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [14]:
rnn_cnn2=CNN_GRU()
rnn_cnn2.initialize_path(project_folder)
rnn_cnn2.initialize_image_properties(image_height=120,image_width=120)
rnn_cnn2.initialize_hyperparams(frames_to_sample=18,batch_size=50,num_epochs=30)
rnn_cnn2_model=rnn_cnn2.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
rnn_cnn2_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 18, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 18, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 18, 30, 30, 64)   

In [15]:
print("Total Params:", rnn_cnn2_model.count_params())
history_model17=rnn_cnn2.train_model(rnn_cnn2_model,augment_data=True)

Total Params: 2573925
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
14/14 [==============================] - ETA: 0s - loss: 1.4487 - categorical_accuracy: 0.3771 
Epoch 00001: saving model to model_init_2020-11-0909_22_09.410844/model-00001-1.44875-0.37707-1.58272-0.23000.h5
14/14 [==============================] - 301s 21s/step - loss: 1.4487 - categorical_accuracy: 0.3771 - val_loss: 1.5827 - val_categorical_accuracy: 0.2300
Epoch 2/30
14/14 [==============================] - ETA: 0s - loss: 0.9184 - categorical_accuracy: 0.6433 
Epoch 00002: saving model to model_init_2020-11-0909_22_09.410844/model-00002-0.91843-0.64329-1.60565-0.32000.h5
14/14 [==============================] - 300s 21s/step - loss: 0.9184 - categorical_accuracy: 0.6433 - val_loss: 1.6057 - val_categorical_accuracy: 0.3200
Epoch 3/30
14/14 [==============================] - ETA: 0s - loss: 0.7073 - categorical_accuracy: 0.7519 
Epoch 00003: saving model to model_init_2020-

Epoch 21/30
14/14 [==============================] - ETA: 0s - loss: 0.1680 - categorical_accuracy: 0.9811 
Epoch 00021: saving model to model_init_2020-11-0909_22_09.410844/model-00021-0.16801-0.98115-2.52115-0.25000.h5

Epoch 00021: ReduceLROnPlateau reducing learning rate to 6.399999961104187e-08.
14/14 [==============================] - 315s 22s/step - loss: 0.1680 - categorical_accuracy: 0.9811 - val_loss: 2.5211 - val_categorical_accuracy: 0.2500
Epoch 22/30
14/14 [==============================] - ETA: 0s - loss: 0.1587 - categorical_accuracy: 0.9796 
Epoch 00022: saving model to model_init_2020-11-0909_22_09.410844/model-00022-0.15867-0.97964-2.18661-0.35000.h5
14/14 [==============================] - 306s 22s/step - loss: 0.1587 - categorical_accuracy: 0.9796 - val_loss: 2.1866 - val_categorical_accuracy: 0.3500
Epoch 23/30
14/14 [==============================] - ETA: 0s - loss: 0.1719 - categorical_accuracy: 0.9781 
Epoch 00023: saving model to model_init_2020-11-0909_22_09.

In [ ]:
#Lowering the Batch Size and checking if it it improving the accuracy

In [16]:
rnn_cnn2=CNN_GRU()
rnn_cnn2.initialize_path(project_folder)
rnn_cnn2.initialize_image_properties(image_height=120,image_width=120)
rnn_cnn2.initialize_hyperparams(frames_to_sample=18,batch_size=30,num_epochs=30)
rnn_cnn2_model=rnn_cnn2.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
rnn_cnn2_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_13 (TimeDis (None, 18, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_14 (TimeDis (None, 18, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_15 (TimeDis (None, 18, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_16 (TimeDis (None, 18, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_17 (TimeDis (None, 18, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_18 (TimeDis (None, 18, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 18, 30, 30, 64)   

In [17]:
print("Total Params:", rnn_cnn2_model.count_params())
history_model17=rnn_cnn2.train_model(rnn_cnn2_model,augment_data=True)

Total Params: 2573925
Epoch 1/30
23/23 [==============================] - ETA: 0s - loss: 1.4728 - categorical_accuracy: 0.3612 
Epoch 00001: saving model to model_init_2020-11-0912_06_38.151126/model-00001-1.47275-0.36124-1.69419-0.21000.h5
23/23 [==============================] - 311s 14s/step - loss: 1.4728 - categorical_accuracy: 0.3612 - val_loss: 1.6942 - val_categorical_accuracy: 0.2100
Epoch 2/30
23/23 [==============================] - ETA: 0s - loss: 0.9660 - categorical_accuracy: 0.6275 
Epoch 00002: saving model to model_init_2020-11-0912_06_38.151126/model-00002-0.96599-0.62745-2.11128-0.20000.h5
23/23 [==============================] - 310s 13s/step - loss: 0.9660 - categorical_accuracy: 0.6275 - val_loss: 2.1113 - val_categorical_accuracy: 0.2000
Epoch 3/30
23/23 [==============================] - ETA: 0s - loss: 0.7147 - categorical_accuracy: 0.7572 
Epoch 00003: saving model to model_init_2020-11-0912_06_38.151126/model-00003-0.71467-0.75716-2.06322-0.24000.h5
23/23 [=

Epoch 22/30
23/23 [==============================] - ETA: 0s - loss: 0.1394 - categorical_accuracy: 0.9857 
Epoch 00022: saving model to model_init_2020-11-0912_06_38.151126/model-00022-0.13944-0.98567-1.53549-0.43000.h5
23/23 [==============================] - 308s 13s/step - loss: 0.1394 - categorical_accuracy: 0.9857 - val_loss: 1.5355 - val_categorical_accuracy: 0.4300
Epoch 23/30
23/23 [==============================] - ETA: 0s - loss: 0.1454 - categorical_accuracy: 0.9796 
Epoch 00023: saving model to model_init_2020-11-0912_06_38.151126/model-00023-0.14539-0.97964-1.25033-0.55000.h5
23/23 [==============================] - 308s 13s/step - loss: 0.1454 - categorical_accuracy: 0.9796 - val_loss: 1.2503 - val_categorical_accuracy: 0.5500
Epoch 24/30
23/23 [==============================] - ETA: 0s - loss: 0.1495 - categorical_accuracy: 0.9827 
Epoch 00024: saving model to model_init_2020-11-0912_06_38.151126/model-00024-0.14945-0.98265-1.22451-0.55000.h5
23/23 [====================

In [ ]:
#Reducing the batch size

In [19]:
rnn_cnn2=CNN_GRU()
rnn_cnn2.initialize_path(project_folder)
rnn_cnn2.initialize_image_properties(image_height=120,image_width=120)
rnn_cnn2.initialize_hyperparams(frames_to_sample=18,batch_size=20,num_epochs=30)
rnn_cnn2_model=rnn_cnn2.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
rnn_cnn2_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_26 (TimeDis (None, 18, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_27 (TimeDis (None, 18, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_28 (TimeDis (None, 18, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_29 (TimeDis (None, 18, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_30 (TimeDis (None, 18, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_31 (TimeDis (None, 18, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_32 (TimeDis (None, 18, 30, 30, 64)   

In [20]:
print("Total Params:", rnn_cnn2_model.count_params())
history_model17=rnn_cnn2.train_model(rnn_cnn2_model,augment_data=True)

Total Params: 2573925
Epoch 1/30
34/34 [==============================] - ETA: 0s - loss: 1.4653 - categorical_accuracy: 0.3891
Epoch 00001: saving model to model_init_2020-11-0916_51_18.184820/model-00001-1.46535-0.38914-1.72832-0.33000.h5
34/34 [==============================] - 317s 9s/step - loss: 1.4653 - categorical_accuracy: 0.3891 - val_loss: 1.7283 - val_categorical_accuracy: 0.3300
Epoch 2/30
34/34 [==============================] - ETA: 0s - loss: 0.8955 - categorical_accuracy: 0.6855
Epoch 00002: saving model to model_init_2020-11-0916_51_18.184820/model-00002-0.89548-0.68552-2.25163-0.21000.h5
34/34 [==============================] - 315s 9s/step - loss: 0.8955 - categorical_accuracy: 0.6855 - val_loss: 2.2516 - val_categorical_accuracy: 0.2100
Epoch 3/30
34/34 [==============================] - ETA: 0s - loss: 0.6173 - categorical_accuracy: 0.7896
Epoch 00003: saving model to model_init_2020-11-0916_51_18.184820/model-00003-0.61731-0.78959-2.76125-0.21000.h5
34/34 [======

Epoch 22/30
34/34 [==============================] - ETA: 0s - loss: 0.0805 - categorical_accuracy: 0.9992
Epoch 00022: saving model to model_init_2020-11-0916_51_18.184820/model-00022-0.08054-0.99925-0.73457-0.73000.h5
34/34 [==============================] - 316s 9s/step - loss: 0.0805 - categorical_accuracy: 0.9992 - val_loss: 0.7346 - val_categorical_accuracy: 0.7300
Epoch 23/30
34/34 [==============================] - ETA: 0s - loss: 0.0823 - categorical_accuracy: 0.9985
Epoch 00023: saving model to model_init_2020-11-0916_51_18.184820/model-00023-0.08230-0.99849-0.57033-0.81000.h5
34/34 [==============================] - 318s 9s/step - loss: 0.0823 - categorical_accuracy: 0.9985 - val_loss: 0.5703 - val_categorical_accuracy: 0.8100
Epoch 24/30
34/34 [==============================] - ETA: 0s - loss: 0.0884 - categorical_accuracy: 0.9932
Epoch 00024: saving model to model_init_2020-11-0916_51_18.184820/model-00024-0.08843-0.99321-0.79388-0.72000.h5
34/34 [=========================

# Transfer Learning

In [21]:
from keras.applications import mobilenet
mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

class CNN_RNN_LSTM_TL(ModelBuilder):
    
    def define_model(self,lstm_cells=64,dense_neurons=64,dropout=0.25):
        
        model = Sequential()
        model.add(TimeDistributed(mobilenet_transfer,input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        
        
        for layer in model.layers:
            layer.trainable = False
        
        
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        model.add(TimeDistributed(Flatten()))

        model.add(LSTM(lstm_cells))
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(Dropout(dropout))
        
        model.add(Dense(self.num_classes, activation='softmax'))
        
        
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [26]:
rnn_cnn_tl=CNN_RNN_LSTM_TL()
rnn_cnn_tl.initialize_path(project_folder)
rnn_cnn_tl.initialize_image_properties(image_height=120,image_width=120)
rnn_cnn_tl.initialize_hyperparams(frames_to_sample=18,batch_size=20,num_epochs=25)
rnn_cnn_tl_model=rnn_cnn_tl.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
rnn_cnn_tl_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_47 (TimeDis (None, 18, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_48 (TimeDis (None, 18, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_49 (TimeDis (None, 18, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_50 (TimeDis (None, 18, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               590336    
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)              

In [27]:
print("Total Params:", rnn_cnn_tl_model.count_params())
history_model17=rnn_cnn2.train_model(rnn_cnn_tl_model,augment_data=True)

Total Params: 3840453
Epoch 1/30
34/34 [==============================] - ETA: 0s - loss: 1.4070 - categorical_accuracy: 0.3952
Epoch 00001: saving model to model_init_2020-11-0919_52_33.773476/model-00001-1.40695-0.39517-1.32648-0.44000.h5
34/34 [==============================] - 190s 6s/step - loss: 1.4070 - categorical_accuracy: 0.3952 - val_loss: 1.3265 - val_categorical_accuracy: 0.4400
Epoch 2/30
34/34 [==============================] - ETA: 0s - loss: 0.7331 - categorical_accuracy: 0.7466
Epoch 00002: saving model to model_init_2020-11-0919_52_33.773476/model-00002-0.73314-0.74661-1.00794-0.64000.h5
34/34 [==============================] - 197s 6s/step - loss: 0.7331 - categorical_accuracy: 0.7466 - val_loss: 1.0079 - val_categorical_accuracy: 0.6400
Epoch 3/30
34/34 [==============================] - ETA: 0s - loss: 0.3533 - categorical_accuracy: 0.8876
Epoch 00003: saving model to model_init_2020-11-0919_52_33.773476/model-00003-0.35331-0.88763-0.83833-0.68000.h5
34/34 [======

Epoch 22/30
34/34 [==============================] - ETA: 0s - loss: 0.0059 - categorical_accuracy: 1.0000
Epoch 00022: saving model to model_init_2020-11-0919_52_33.773476/model-00022-0.00594-1.00000-1.15571-0.69000.h5
34/34 [==============================] - 195s 6s/step - loss: 0.0059 - categorical_accuracy: 1.0000 - val_loss: 1.1557 - val_categorical_accuracy: 0.6900
Epoch 23/30
34/34 [==============================] - ETA: 0s - loss: 0.0059 - categorical_accuracy: 1.0000
Epoch 00023: saving model to model_init_2020-11-0919_52_33.773476/model-00023-0.00585-1.00000-1.08524-0.64000.h5
34/34 [==============================] - 195s 6s/step - loss: 0.0059 - categorical_accuracy: 1.0000 - val_loss: 1.0852 - val_categorical_accuracy: 0.6400
Epoch 24/30
34/34 [==============================] - ETA: 0s - loss: 0.0056 - categorical_accuracy: 1.0000
Epoch 00024: saving model to model_init_2020-11-0919_52_33.773476/model-00024-0.00563-1.00000-1.30749-0.64000.h5

Epoch 00024: ReduceLROnPlateau 

# Transfer Learning GRU

In [10]:
from keras.applications import mobilenet

mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

class CNN_RNN_GRU_TL(ModelBuilder):
    
    def define_model(self,gru_cells=64,dense_neurons=64,dropout=0.25):
        
        model = Sequential()
        model.add(TimeDistributed(mobilenet_transfer,input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
 
        
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        model.add(TimeDistributed(Flatten()))

        model.add(GRU(gru_cells))
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(Dropout(dropout))
        
        model.add(Dense(self.num_classes, activation='softmax'))
        
        
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [11]:
rnn_cnn_gru_tl=CNN_RNN_GRU_TL()
rnn_cnn_gru_tl.initialize_path(project_folder)
rnn_cnn_gru_tl.initialize_image_properties(image_height=120,image_width=120)
rnn_cnn_gru_tl.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=20)
rnn_cnn_gru_tl_model=rnn_cnn_gru_tl.define_model(gru_cells=128,dense_neurons=128,dropout=0.25)
rnn_cnn_gru_tl_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 16, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 16, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 16, 1024)          0         
_________________________________________________________________
gru (GRU)                    (None, 128)               443136    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

In [ ]:
print("Total Params:",rnn_cnn_gru_tl_model.count_params())
history_rnn_cnn_gru_tl=rnn_cnn_gru_tl.train_model(rnn_cnn_gru_tl_model,augment_data=True)

Total Params: 3693253
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.1317 - categorical_accuracy: 0.5362 
Epoch 00001: saving model to model_init_2020-11-0921_44_10.425982/model-00001-1.13167-0.53620-1.07881-0.55000.h5
34/34 [==============================] - 1014s 30s/step - loss: 1.1317 - categorical_accuracy: 0.5362 - val_loss: 1.0788 - val_categorical_accuracy: 0.5500
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 0.3538 - categorical_accuracy: 0.8688 
Epoch 00002: saving model to model_init_2020-11-0921_44_10.425982/model-00002-0.35384-0.86878-0.47294-0.86000.h5
34/34 [==============================] - 1176s 35s/step - loss: 0.3538 - categorical_accuracy: 0.8688 - val_loss: 0.4729 - val_categorical_accuracy: 0.8600
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 0.3263 - categorical_accuracy: 0.8914 
Epoch 00003: saving model to model_init_202